\FloatBarrier

<!-- \setcounter{section}{0} -->
\renewcommand{\thesection}{\Alph{section}}

<!-- \setcounter{table}{0} -->
\renewcommand{\thetable}{A\arabic{table}}

<!-- \setcounter{figure}{0} -->
\renewcommand{\thefigure}{A\arabic{figure}}

<!-- # Supplementary Material {.appendix} -->


# Notation {.appendix}

- $y^+$: The target class and also the index of the target class.
- $y^-$: The non-target class and also the index of non-the target class.
- $\mathbf{y}^+$: The one-hot encoded output vector for the target class. 
- $\theta$: Model parameters (unspecified).
- $\Theta$: Matrix of parameters. 



# Technical Details of Our Approach {.appendix} 


## Generating Counterfactuals through Gradient Descent {#sec-app-ce}

In this section, we provide some background on gradient-based counterfactual generators (@sec-app-ce-background) and discuss how we define convergence in this context (@sec-app-conv).

### Background {#sec-app-ce-background}

Gradient-based counterfactual search was originally proposed by @wachter2017counterfactual. It generally solves the following unconstrained objective,

$$
\begin{aligned}
\min_{\mathbf{z}^\prime \in \mathcal{Z}^L} \left\{  {\text{yloss}(\mathbf{M}_{\theta}(g(\mathbf{z}^\prime)),\mathbf{y}^+)}+ \lambda {\text{cost}(g(\mathbf{z}^\prime)) }  \right\} 
\end{aligned} 
$$

where $g: \mathcal{Z} \mapsto \mathcal{X}$ is an invertible function that maps from the $L$-dimensional counterfactual state space to the feature space and $\text{cost}(\cdot)$ denotes one or more penalties that are used to induce certain properties of the counterfactual outcome. As above, $\mathbf{y}^+$ denotes the target output and $\mathbf{M}_{\theta}(\mathbf{x})$ returns the logit predictions of the underlying classifier for $\mathbf{x}=g(\mathbf{z})$.

For all generators used in this work we use standard logit crossentropy loss for $\text{yloss}(\cdot)$. All generators also penalize the distance ($\ell_1$-norm) of counterfactuals from their original factual state. For *Generic* and *ECCo*, we have $\mathcal{Z}:=\mathcal{X}$ and $g(\mathbf{z})=g(\mathbf{z})^{-1}=\mathbf{z}$, that is counterfactual are searched directly in the feature space. Conversely, *REVISE* traverses the latent space of a variational autoencoder (VAE) fitted to the training data, where $g(\cdot)$ corresponds to the decoder [@joshi2019realistic]. In addition to the distance penalty, *ECCo* uses an additional penalty component that regularizes the energy associated with the counterfactual, $\mathbf{x}^\prime$ [@altmeyer2024faithful]. 

### Convergence {#sec-app-conv}

An important consideration when generating counterfactual explanations using gradient-based methods is how to define convergence. Two common choices are to 1) perform gradient descent over a fixed number of iterations $T$, or 2) conclude the search as soon as the predicted probability for the target class has reached a pre-determined threshold, $\tau$: $\mathcal{S}(\mathbf{M}_\theta(\mathbf{x}^\prime))[y^+] \geq \tau$. We prefer the latter for our purposes, because it explicitly defines convergence in terms of the black-box model, $\mathbf{M}(\mathbf{x})$.

Defining convergence in this way allows for a more intuitive interpretation of the resulting counterfactual outcomes than with fixed $T$. Specifically, it allows us to think of counterfactuals as explaining 'high-confidence' predictions by the model for the target class $y^+$. Depending on the context and application, different choices of $\tau$ can be considered as representing 'high-confidence' predictions.


In [ ]:
#| eval: false

projectdir = splitpath(pwd()) |>
    ss -> joinpath(ss[1:findall([s == "CounterfactualTraining.jl" for s in ss])[1]]...) 
cd(projectdir)

using CTExperiments
using CTExperiments.CounterfactualExplanations
using CTExperiments.CounterfactualTraining
using CTExperiments.Flux
using CTExperiments.Plots
using CTExperiments.TaijaPlotting
using Plots.PlotMeasures

## Protecting Mutability Constraints with Linear Classifiers {#sec-app-constraints}

In @sec-constraints we explain that to avoid penalizing implausibility that arises due to mutability constraints, we impose a point mass prior on $p(\mathbf{x})$ for the corresponding feature. We argue in @sec-constraints that this approach induces models to be less sensitive to immutable features and demonstrate this empirically in @sec-experiments. Below we derive the analytical results in @prp-mtblty.

::: {.proof}

Let $d_{\text{mtbl}}$ and $d_{\text{immtbl}}$ denote some mutable and immutable feature, respectively. Suppose that $\mu_{y^-,d_{\text{immtbl}}} < \mu_{y^+,d_{\text{immtbl}}}$ and $\mu_{y^-,d_{\text{mtbl}}} > \mu_{y^+,d_{\text{mtbl}}}$, where $\mu_{k,d}$ denotes the conditional sample mean of feature $d$ in class $k$. In words, we assume that the immutable feature tends to take lower values for samples in the non-target class $y^-$ than in the target class $y^+$. We assume the opposite to hold for the mutable feature.

Assuming multivariate Gaussian class densities with common diagonal covariance matrix $\Sigma_k=\Sigma$ for all $k \in \mathcal{K}$, we have for the log likelihood ratio between any two classes $k,m \in \mathcal{K}$ [@hastie2009elements]:

$$
\log \frac{p(k|\mathbf{x})}{p(m|\mathbf{x})}=\mathbf{x}^\intercal \Sigma^{-1}(\mu_{k}-\mu_{m})  + \text{const}
$$ {#eq-loglike}

By independence of $x_1,...,x_D$, the full log-likelihood ratio decomposes into:

$$
\log \frac{p(k|\mathbf{x})}{p(m|\mathbf{x})} = \sum_{d=1}^D \frac{\mu_{k,d}-\mu_{m,d}}{\sigma_{d}^2} x_{d} + \text{const}
$$ {#eq-loglike-decomp}

By the properties of our classifier (*multinomial logistic regression*), we have:

$$
\log \frac{p(k|\mathbf{x})}{p(m|\mathbf{x})} = \sum_{d=1}^D \left( \theta_{k,d} - \theta_{m,d} \right)x_d + \text{const}
$$ {#eq-multi}

where $\theta_{k,d}=\Theta[k,d]$ denotes the coefficient on feature $d$ for class $k$. 

Based on @eq-loglike-decomp and @eq-multi we can identify that $(\mu_{k,d}-\mu_{m,d}) \propto (\theta_{k,d} - \theta_{m,d})$ under the assumptions we made above. Hence, we have that $(\theta_{y^-,d_{\text{immtbl}}} - \theta_{y^+,d_{\text{immtbl}}}) < 0$ and $(\theta_{y^-,d_{\text{mtbl}}} - \theta_{y^+,d_{\text{mtbl}}}) > 0$

Let $\mathbf{x}^\prime$ denote some randomly chosen individual from class $y^-$ and let $y^+ \sim p(y)$ denote the randomly chosen target class. Then the partial derivative of the contrastive divergence penalty [@eq-div] with respect to coefficient $\theta_{y^+,d}$ is equal to 

$$
\frac{\partial}{\partial\theta_{y^+,d}} \left(\text{div}(\mathbf{x},\mathbf{x^\prime},\mathbf{y};\theta)\right) = \frac{\partial}{\partial\theta_{y^+,d}} \left( \left(-\mathbf{M}_\theta(\mathbf{x})[y^+]\right) - \left(-\mathbf{M}_\theta(\mathbf{x}^\prime)[y^+]\right) \right) = x_{d}^\prime - x_{d}
$$ {#eq-grad}

and equal to zero everywhere else.

Since $(\mu_{y^-,d_{\text{immtbl}}} < \mu_{y^+,d_{\text{immtbl}}})$ we are more likely to have $(x_{d_{\text{immtbl}}}^\prime - x_{d_{\text{immtbl}}}) < 0$ than vice versa at initialization. Similarly, we are more likely to have $(x_{d_{\text{mtbl}}}^\prime - x_{d_{\text{mtbl}}}) > 0$ since $(\mu_{y^-,d_{\text{mtbl}}} > \mu_{y^+,d_{\text{mtbl}}})$.

This implies that if we do not protect feature $d_{\text{immtbl}}$, the contrastive divergence penalty will decrease $\theta_{y^-,d_{\text{immtbl}}}$ thereby exacerbating the existing effect $(\theta_{y^-,d_{\text{immtbl}}} - \theta_{y^+,d_{\text{immtbl}}}) < 0$. In words, not protecting the immutable feature would have the undesirable effect of making the classifier more sensitive to this feature, in that it would be more likely to predict class $y^-$ as opposed to $y^+$ for lower values of $d_{\text{immtbl}}$. 

By the same rationale, the contrastive divergence penalty can generally be expected to increase $\theta_{y^-,d_{\text{mtbl}}}$ exacerbating $(\theta_{y^-,d_{\text{mtbl}}} - \theta_{y^+,d_{\text{mtbl}}}) > 0$. In words, this has the effect of making the classifier more sensitive to the mutable feature, in that it would be more likely to predict class $y^-$ as opposed to $y^+$ for higher values of $d_{\text{mtbl}}$.

Thus, our proposed approach of protecting feature $d_{\text{immtbl}}$ has the net affect of decreasing the classifier's sensitivity to the immutable feature relative to the mutable feature (i.e. no change in sensitivity for $d_{\text{immtbl}}$ relative to increased sensitivity for $d_{\text{mtbl}}$).

:::

In [ ]:
#| eval: false

using CTExperiments: 
    get_ce_data, 
    train_val_split, 
    build_model, 
    LinearModel, 
    MLPModel,
    get_input_encoder

# Callback:
using CTExperiments: get_log_reg_params, get_decision_boundary

function plot_db(model, ces; _xlab="Debt", _ylab="Age", plot_contour::Bool=false)

    x = (ce -> ce.counterfactual).(ces) |> xs -> reduce(hcat, xs)
    x0 = (ce -> ce.factual).(ces) |> xs -> reduce(hcat, xs)

    xlab = _constraint[1] == "both" ? "$(_xlab) (mutable)" : "$(_xlab) (immutable)"
    ylab = _constraint[2] == "both" ? "$(_ylab) (mutable)" : "$(_ylab) (immutable)"

    # Data and decision boundary:
    plt = Plots.scatter(
        ce_data, 
        # label=["1=Default" "2=No Default"], 
        # legend_position=:topright, 
        xlab=xlab, ylab=ylab,  
        axis=nothing, 
        legend=false,
        title=_title
    )
    if length(model) == 1
        coeff = get_log_reg_params(model)
        db = get_decision_boundary(coeff)
        Plots.abline!(plt, db.slope, db.intercept; lw=5, label="Dec. Boundary")
    end

    # Counterfactuals:
    target = 2
    factual = 1
    if !any(isnothing.(x))
        yhat = [argmax(y) for y in eachcol(model(x))]
        yhat0 = [argmax(y) for y in eachcol(model(x0))]
        idx_plotted = yhat0.==factual .|| yhat0.==target
        if any(idx_plotted)

            # # Paths:
            # u = []
            # v = []
            # for (i,ce) in enumerate(eachcol(x))
            #     Δ = ce - x0[:,i]
            #     push!(u, Δ[1])
            #     push!(v, Δ[2])
            # end
            # Plots.quiver!(x0[1,idx_plotted], x0[2,idx_plotted], quiver=(u[idx_plotted], v[idx_plotted]), color=yhat0[idx_plotted])
            
            # End points:
            Plots.scatter!(x[1,idx_plotted], x[2,idx_plotted], label=["CE (y⁺=1)" "CE (y⁺=2)"], ms=15, shape=:star, color=yhat[idx_plotted], group=yhat[idx_plotted], mscolor=yhat0[idx_plotted])

        end
    end
    display(plt)
end

# Data:
specs = [
    # ("(a)", VanillaObjective(needs_ce=true), ["both", "both"]),
    # ("(b)", FullObjective(lambda=[1.0,0.5,0.01,0.1]), ["both", "both"]),
    # ("(c)", VanillaObjective(needs_ce=true), ["both", "none"]),
    ("(d)", FullObjective(lambda=[1.0,0.5,0.01,0.1]), ["both", "none"]),
]

In [ ]:
#| eval: false

using CounterfactualExplanations.Convergence
using Random
Random.seed!(42)

models = []
plts = []

for (title, obj, constraint) in specs

    # Globals for the callback:
    global _title = title
    global _constraint = constraint

    # Data:
    data = LinearlySeparable(
        n_train=3000,
        batchsize=50,
        mutability=constraint
    )
    global ce_data = get_ce_data(data)
    val_size = data.n_validation / (data.n_validation + data.n_train)
    train_set, val_set, _ = train_val_split(data, ce_data, val_size)

    # Model:
    nin = size(first(train_set)[1], 1)
    nout = size(first(train_set)[2], 1)
    model = build_model(MLPModel(), nin, nout)

    # Objective:
    opt = AMSGrad()
    generator = GenericGenerator(opt=Descent(0.1))
    opt_state = Flux.setup(opt, model)
    conv = DecisionThresholdConvergence(decision_threshold=0.75, max_iter=30)

    model, logs = counterfactual_training(
        obj,
        model,
        generator,
        train_set,
        opt_state;
        val_set = val_set,
        nepochs = 100,
        mutability = Symbol.(constraint),
        callback = plot_db,
        nce=1000,
        convergence=conv, 
        verbose=3
    )

    push!(models, model)
    push!(plts, current())
end

In [ ]:
#| eval: false

plt = Plots.plot(
    plts..., 
    layout=(1,4), 
    size=(1150,250),  
    left_margin = 10mm, 
    bottom_margin = 5mm,
    top_margin = 3mm,
    right_margin = 10mm,
)
display(plt)
Plots.savefig(plt, "paper/figures/poc.svg")

In [ ]:
#| eval: false

plts = []
titles = ["(a)", "(b)"]
_xlab = "Existing Debt"
_ylab = "Age"
for (i,model) in enumerate(models)
    coeff = get_log_reg_params(model)
    db = get_decision_boundary(coeff)
    xlab = constraints[i][1] == "both" ? "$(_xlab) (mutable)" : "$(_xlab) (immutable)"
    ylab = constraints[i][2] == "both" ? "$(_ylab) (mutable)" : "$(_ylab) (immutable)"
    plt = Plots.scatter(ce_data; xlab=xlab, ylab=ylab, bottom_margin = 5mm, left_margin = 5mm, label=["Default" "No Default"], title=titles[i])
    Plots.abline!(plt, db.slope, db.intercept; lw=5, label="Dec. Boundary")

    push!(plts, plt)
end

plt = Plots.plot(plts..., layout=(1,2), size=(600,300))
Plots.savefig(plt, "paper/figures/app_mtblty.svg")

## Domain Constraints

We apply domain constraints on counterfactuals during training and evaluation. There are at least two good reasons for doing so. Firstly, within the context of explainability and algorithmic recourse, real-world attributes are often domain constrained: the *age* feature, for example, is lower bounded by zero and upper bounded by the maximum human lifespan. Secondly, domain constraints help mitigate training instabilities commonly associated with energy-based modelling [@grathwohl2020your;@altmeyer2024faithful].

For our image datasets, features are pixel values and hence the domain is constrained by the lower and upper bound of values that pixels can take depending on how they are scaled (in our case $[-1,1]$). For all other features $d$ in our synthetic and tabular datasets, we automatically infer domain constraints $[x_d^{\text{LB}},x_d^{\text{UB}}]$  as follows,

$$
\begin{aligned}
x_d^{\text{LB}} &= \arg\min_{x_d} \{\mu_d - n_{\sigma_d}\sigma_d, \arg \min_{x_d} x_d\} \\
x_d^{\text{UB}} &= \arg\max_{x_d} \{\mu_d + n_{\sigma_d}\sigma_d, \arg \max_{x_d} x_d\} 
\end{aligned}
$$ {#eq-domain}

where $\mu_d$ and $\sigma_d$ denote the sample mean and standard deviation of feature $d$. We set $n_{\sigma_d}=3$ across the board but higher values and hence wider bounds may be appropriate depending on the application.


## Training Details {#sec-app-training}

In this section, we describe the training procedure in detail. While the details laid out here are not crucial for understanding our proposed approach, they are of importance to anyone looking to implement counterfactual training. 




# Detailed Results {.appendix}

## Qualitative Findings for Image Data

::: {.callout-note}

@fig-mnist shows much more plausible (faithful) counterfactuals for a model with CT than the model with conventional training (@fig-mnist-vanilla). In fact, this is not even using *ECCo+* and still showing better results than the best results we achieved in our AAAI paper for JEM ensembles.

:::

![Counterfactual images for *MLP* with counterfactual training. The underlying generator, *ECCo*, aims to generate counterfactuals that are faithful to the model [@altmeyer2024faithful].](/paper/figures/mnist_mlp.png){#fig-mnist}

![Counterfactual images for *MLP* with conventional training. The underlying generator, *ECCo*, aims to generate counterfactuals that are faithful to the model [@altmeyer2024faithful].](/paper/figures/mnist_mlp_vanilla.png){#fig-mnist-vanilla}


## Grid Searches {#sec-app-grid}

In [ ]:
projectdir = splitpath(pwd()) |>
    ss -> joinpath(ss[1:findall([s == "CounterfactualTraining.jl" for s in ss])[1]]...) 
cd(projectdir)

using CTExperiments
using CTExperiments.CSV
using CTExperiments.DataFrames
using CTExperiments.StatsBase

using DotEnv
DotEnv.load!()

In [ ]:
res_dir = ENV["FINAL_GRID_RESULTS"]

To assess the hyperparameter sensitivity of our proposed training regime we ran multiple large grid searches for all of our synthetic datasets. We have grouped these grid searches into multiple categories: 

1. **Generator Parameters** (@sec-app-grid-gen): Investigates the effect of changing hyperparameters that affect the counterfactual outcomes during the training phase.
2. **Penalty Strengths** (@sec-app-grid-pen): Investigates the effect of changing the penalty strengths in out proposed objective (@eq-obj).
3. **Other Parameters** (@sec-app-grid-train): Investigates the effect of changing other training parameters, including the total number of generated counterfactuals in each epoch.

We begin by summarizing the high-level findings in @sec-app-grid-hl. For each of the categories, @sec-app-grid-gen to @sec-app-grid-train then present all details including the exact parameter grids, average predictive performance outcomes and key evaluation metrics for the generated counterfactuals. 

### Evaluation Details

To measure predictive performance, we compute the accuracy and F1-score for all models on test data (@tbl-acc-gen, @tbl-acc-pen, @tbl-acc-train). 

We evaluated 

### High-Level Findings {#sec-app-grid-hl}

Overall, we find that Counterfactual Training (CT) achieves it key objectives consistently across all hyperparameter settings and also broadly across datasets: plausibility is improved  We do observe strong sensitivity to certain hyperparameters, but clear and manageable patterns emerge in those cases. With respect to the underlying training data distribution, we find that outcomes for CT seem to depend to some degree on class separability.

#### Predictive Performance

We find that CT is associated with little to no decrease in average predictive performance for our synthetic datasets: test accuracy and F1-scores decrease by at most ~1 percentage point, but generally much less (@tbl-acc-gen, @tbl-acc-pen, @tbl-acc-train). Variation across hyperparameters is negligible as indicated by small standard deviations for these metrics across the board. 

#### Key Counterfactual Outcomes



### Generator Parameters {#sec-app-grid-gen}

In [ ]:
grid_dir = joinpath(res_dir, "gen_params/mlp")
data_dirs = readdir(grid_dir) |> x -> joinpath.(grid_dir, x) |> x -> x[isdir.(x)]
eval_grids = (p -> EvaluationGrid(joinpath(p,"grid_config.toml"))).(data_dirs)
data_names = basename.(data_dirs)
suffix = "evaluation/results/ce/decision_threshold_exper---lambda_energy_exper---maxiter_exper---maxiter---decision_threshold_exper/"

The hyperparameter grid with varying generator parameters during training is shown in @nte-gen-params-final-run-train. The corresponding evaluation grid used for these experiments is shown in @nte-gen-params-final-run-eval.

::: {#nte-gen-params-final-run-train .callout-note}

## Training Phase

In [ ]:
#| output: asis
dict = CTExperiments.from_toml(joinpath(grid_dir, "lin_sep/grid_config.toml")) 
println(CTExperiments.dict_to_quarto_markdown(dict))

:::

::: {#nte-gen-params-final-run-eval .callout-note}

## Evaluation Phase

In [ ]:
#| output: asis
dict = CTExperiments.from_toml(joinpath(grid_dir, "lin_sep/evaluation/evaluation_grid_config.toml"))
println(CTExperiments.dict_to_quarto_markdown(dict))

:::

#### Accuracy

::: {#tbl-acc-gen}

::: {.content-hidden unless-format="pdf"}

In [ ]:
#| output: asis
df = CTExperiments.aggregate_performance(eval_grids; byvars=["objective"]) 
get_table_inputs(df, "mean";backend=Val(:latex)) |>
    inputs -> tabulate_results(inputs; wrap_table=false)

:::

Predictive performance measures by dataset and objective averaged across training-phase parameters (@nte-gen-params-final-run-train) and evaluation-phase parameters (@nte-gen-params-final-run-eval).

:::

#### Plausibility

In [ ]:
#| output: asis

fig_label_prefix = "grid-gen_params-plaus"
fig_labels = (nm -> "fig-$(fig_label_prefix)-$nm").(data_names)
_str = "The results with respect to the plausibility measure are shown in @$(fig_labels[1]) to @$(fig_labels[end])."
println(_str)

In [ ]:
#| output: asis
 
imgfname = "plausibility_distance_from_target.png"
fig_caption = "Average outcomes for the plausibility measure across hyperparameters."
full_paths = joinpath.(data_dirs, joinpath(suffix,imgfname))
include_img_commands = CTExperiments.get_img_command(data_names, full_paths, fig_labels; fig_caption) 
_str = join(include_img_commands, "\n\n")
println(_str)

#### Cost

In [ ]:
#| output: asis

fig_label_prefix = "grid-gen_params-cost"
fig_labels = (nm -> "fig-$(fig_label_prefix)-$nm").(data_names)
_str = "The results with respect to the cost measure are shown in @$(fig_labels[1]) to @$(fig_labels[end])."
println(_str)

In [ ]:
#| output: asis
 
imgfname = "distance.png"
fig_caption = "Average outcomes for the cost measure across hyperparameters."
full_paths = joinpath.(data_dirs, joinpath(suffix,imgfname))
include_img_commands = CTExperiments.get_img_command(data_names, full_paths, fig_labels; fig_caption)
_str = join(include_img_commands, "\n\n")
println(_str)

### Penalty Strengths {#sec-app-grid-pen}

In [ ]:
grid_dir = joinpath(res_dir, "penalties/mlp")
data_dirs = readdir(grid_dir) |> x -> joinpath.(grid_dir, x) |> x -> x[isdir.(x)]
eval_grids = (p -> EvaluationGrid(joinpath(p,"grid_config.toml"))).(data_dirs)
data_names = basename.(data_dirs)
suffix = "evaluation/results/ce/lambda_adversarial---lambda_energy_reg---lambda_energy_diff---lambda_adversarial---lambda_adversarial/"

The hyperparameter grid with varying penalty strengths during training is shown in @nte-pen-final-run-train. The corresponding evaluation grid used for these experiments is shown in @nte-pen-final-run-eval.

::: {#nte-pen-final-run-train .callout-note}

## Training Phase

In [ ]:
#| output: asis
dict = CTExperiments.from_toml(joinpath(grid_dir, "lin_sep/grid_config.toml")) 
println(CTExperiments.dict_to_quarto_markdown(dict))

:::

::: {#nte-pen-final-run-eval .callout-note}

## Evaluation Phase

In [ ]:
#| output: asis
dict = CTExperiments.from_toml(joinpath(grid_dir, "lin_sep/evaluation/evaluation_grid_config.toml"))
println(CTExperiments.dict_to_quarto_markdown(dict))

:::

#### Accuracy

::: {#tbl-acc-pen}

::: {.content-hidden unless-format="pdf"}

In [ ]:
#| output: asis
df = CTExperiments.aggregate_performance(eval_grids; byvars=["objective"]) 
get_table_inputs(df, "mean";backend=Val(:latex)) |>
    inputs -> tabulate_results(inputs; wrap_table=false)

:::

Predictive performance measures by dataset and objective averaged across training-phase parameters (@nte-pen-final-run-train) and evaluation-phase parameters (@nte-pen-final-run-eval).

:::

#### Plausibility

In [ ]:
#| output: asis

fig_label_prefix = "grid-pen-plaus"
fig_labels = (nm -> "fig-$(fig_label_prefix)-$nm").(data_names)
_str = "The results with respect to the plausibility measure are shown in @$(fig_labels[1]) to @$(fig_labels[end])."
println(_str)

In [ ]:
#| output: asis
 
imgfname = "plausibility_distance_from_target.png"
fig_caption = "Average outcomes for the plausibility measure across hyperparameters."
full_paths = joinpath.(data_dirs, joinpath(suffix,imgfname))
include_img_commands = CTExperiments.get_img_command(data_names, full_paths, fig_labels; fig_caption) 
_str = join(include_img_commands, "\n\n")
println(_str)

#### Cost

In [ ]:
#| output: asis

fig_label_prefix = "grid-pen-cost"
fig_labels = (nm -> "fig-$(fig_label_prefix)-$nm").(data_names)
_str = "The results with respect to the cost measure are shown in @$(fig_labels[1]) to @$(fig_labels[end])."
println(_str)

In [ ]:
#| output: asis
 
imgfname = "distance.png"
fig_caption = "Average outcomes for the cost measure across hyperparameters."
full_paths = joinpath.(data_dirs, joinpath(suffix,imgfname))
include_img_commands = CTExperiments.get_img_command(data_names, full_paths, fig_labels; fig_caption)
_str = join(include_img_commands, "\n\n")
println(_str)

### Other Parameters {#sec-app-grid-train}

In [ ]:
grid_dir = joinpath(res_dir, "training_params/mlp")
data_dirs = readdir(grid_dir) |> x -> joinpath.(grid_dir, x) |> x -> x[isdir.(x)]
eval_grids = (p -> EvaluationGrid(joinpath(p,"grid_config.toml"))).(data_dirs)
data_names = basename.(data_dirs)
suffix = "evaluation/results/ce/burnin---nce---nepochs---burnin---burnin/"

The hyperparameter grid with other varying training parameters is shown in @nte-train-final-run-train. The corresponding evaluation grid used for these experiments is shown in @nte-train-final-run-eval.

::: {#nte-train-final-run-train .callout-note}

## Training Phase

In [ ]:
#| output: asis
dict = CTExperiments.from_toml(joinpath(grid_dir, "lin_sep/grid_config.toml")) 
println(CTExperiments.dict_to_quarto_markdown(dict))

:::

::: {#nte-train-final-run-eval .callout-note}

## Evaluation Phase

In [ ]:
#| output: asis
dict = CTExperiments.from_toml(joinpath(grid_dir, "lin_sep/evaluation/evaluation_grid_config.toml"))
println(CTExperiments.dict_to_quarto_markdown(dict))

:::

#### Accuracy

::: {#tbl-acc-train}

::: {.content-hidden unless-format="pdf"}

In [ ]:
#| output: asis
df = CTExperiments.aggregate_performance(eval_grids; byvars=["objective"]) 
get_table_inputs(df, "mean";backend=Val(:latex)) |>
    inputs -> tabulate_results(inputs; wrap_table=false)

:::

Predictive performance measures by dataset and objective averaged across training-phase parameters (@nte-train-final-run-train) and evaluation-phase parameters (@nte-train-final-run-eval).

:::

#### Plausibility

In [ ]:
#| output: asis

fig_label_prefix = "grid-train-plaus"
fig_labels = (nm -> "fig-$(fig_label_prefix)-$nm").(data_names)
_str = "The results with respect to the plausibility measure are shown in @$(fig_labels[1]) to @$(fig_labels[end])."
println(_str)

In [ ]:
#| output: asis
 
imgfname = "plausibility_distance_from_target.png"
fig_caption = "Average outcomes for the plausibility measure across hyperparameters."
full_paths = joinpath.(data_dirs, joinpath(suffix,imgfname))
include_img_commands = CTExperiments.get_img_command(data_names, full_paths, fig_labels; fig_caption) 
_str = join(include_img_commands, "\n\n")
println(_str)

#### Cost

In [ ]:
#| output: asis

fig_label_prefix = "grid-train-cost"
fig_labels = (nm -> "fig-$(fig_label_prefix)-$nm").(data_names)
_str = "The results with respect to the cost measure are shown in @$(fig_labels[1]) to @$(fig_labels[end])."
println(_str)

In [ ]:
#| output: asis
 
imgfname = "distance.png"
fig_caption = "Average outcomes for the cost measure across hyperparameters."
full_paths = joinpath.(data_dirs, joinpath(suffix,imgfname))
include_img_commands = CTExperiments.get_img_command(data_names, full_paths, fig_labels; fig_caption)
_str = join(include_img_commands, "\n\n")
println(_str)

<!-- {{< include /paper/sections/other/initial_grids.qmd >}} -->


# Computation Details {.appendix}

<!-- [@DHPC2022] -->
## Hardware

We performed our experiments on a high-performance cluster. Details about the cluster will be disclosed upon publication to avoid revealing information that might interfere with the double-blind review process. Since our experiments involve highly parallel tasks and rather small models by today's standard, we have relied on distributed computing across multiple central processing units (CPU). Graphical processing units (GPU) were not required. Model training for the largest grid searches with 270 unique parameter combinations was parallelized across 34 CPUs with 2GB memory each. The time to completion varied by dataset for reasons discussed in @sec-discussion: 0h49m (*Moons*), 1h4m (*Linearly Separable*), 1h49m (*Circles*), 3h52m (*Overlapping*). Model evaluations for large grid searches were parallelized across 20 CPUs with 3GB memory each. Evaluations for all data sets took less than one hour (<1h) to complete. 

## Software

All computations were performed in the Julia Programming Language [@bezanson2017julia]. We have developed a package for counterfactual training that leverages and extends the functionality provided by several existing packages, most notably [CounterfactualExplanations.jl](https://github.com/JuliaTrustworthyAI/CounterfactualExplanations.jl) [@altmeyer2023explaining] and the [Flux.jl](https://fluxml.ai/Flux.jl/v0.16/) library for deep learning [@innes2018fashionable;@innes2018flux]. For data-wrangling and presentation-ready tables we relied on [DataFrames.jl](https://dataframes.juliadata.org/v1.7/) [@milan2023dataframes] and [PrettyTables.jl](https://ronisbr.github.io/PrettyTables.jl/v2.4/) [@chagas2024pretty], respectively. For plots and visualizations we used both [Plots.jl](https://docs.juliaplots.org/v1.40/) [@PlotsJL] and [Makie.jl](https://docs.makie.org/v0.22/) [@danisch2021makie], in particular [AlgebraOfGraphics.jl](https://aog.makie.org/v0.9.3/). To distribute computational tasks across multiple processors, we have relied on [MPI.jl](https://juliaparallel.org/MPI.jl/v0.20/) [@byrne2021mpi].
